# Tribeca Data Wrangling

This supplemental notebook documents and demonstrates the data wrangling for the Tribeca dataset.

## Sources

Data: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)

Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)


In [47]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data

#data_url = ('https://github.com/LambdaSchool/DS-Unit-2-Linear-Models/blob/master/data/condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

from google.colab import files
uploaded = files.upload()
#df = pd.read_csv(data_url, sep= 'delimeter', header = None)




Saving NYC_Citywide_Rolling_Calendar_Sales.csv to NYC_Citywide_Rolling_Calendar_Sales (2).csv


In [48]:
import io
df = pd.read_csv(io.BytesIO(uploaded['NYC_Citywide_Rolling_Calendar_Sales.csv']))
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,$ - 0,01/01/2019
1,1,FASHION,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,$ - 0,01/01/2019
2,1,FASHION,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,$ - 0,01/01/2019
3,1,GREENWICH VILLAGE-WEST,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,$ - 0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,$ - 0,01/01/2019


In [50]:
# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# Get Pandas Profiling Report
#df.profile_report()
df.head(2)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,$ - 0,01/01/2019
1,1,FASHION,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,$ - 0,01/01/2019


In [42]:
# Does this data include the Tribeca neighborhood?
mask = df['NEIGHBORHOOD'].str.contains('TRIBECA')
df[mask]

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE,SALE_DATE
220,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,222,1013,NaN,R4,"181 HUDSON STREET, 6D",6D,10013.0,1.0,0.0,1.0,"7,878",1840.0,1909.0,2,R4,"$ 2,800,000",01/03/2019,2019-01-03
763,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,9022,NaN,R4,"10 LITTLE WEST STREET, 19A",19A,10004.0,1.0,0.0,1.0,0,1759.0,0.0,2,R4,"$ 2,650,000",01/07/2019,2019-01-07
996,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,212,1404,NaN,R4,"11 BEACH STREET, 1D",1D,10013.0,1.0,0.0,1.0,"11,213",2651.0,1900.0,2,R4,$ - 0,01/08/2019,2019-01-08
1276,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,1441,NaN,R4,"30 LITTLE WEST STREET, 7G",7G,10004.0,1.0,0.0,1.0,0,1190.0,2005.0,2,R4,"$ 1,005,000",01/09/2019,2019-01-09
1542,1,TRIBECA,02 TWO FAMILY DWELLINGS,1,132,26,NaN,S2,"75 WARREN STREET, XX",NaN,10007.0,2.0,1.0,3.0,"1,819",10117.0,1905.0,1,S2,"$ 12,950,000",01/10/2019,2019-01-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22221,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,222,1206,NaN,R4,"443 GREENWICH STREET, 1F",1F,10013.0,1.0,0.0,1.0,"35,127",2429.0,1905.0,2,R4,"$ 5,761,259",04/24/2019,2019-04-24
22732,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,1806,NaN,R4,"70 LITTLE WEST STREET, 11K",11K,10004.0,1.0,0.0,1.0,0,1601.0,2006.0,2,R4,"$ 2,600,000",04/29/2019,2019-04-29
22733,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,8071,NaN,R4,"2 SOUTH END AVENUE, 5G",5G,10280.0,1.0,0.0,1.0,0,634.0,1990.0,2,R4,"$ 605,000",04/29/2019,2019-04-29
22897,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,8132,NaN,R4,"2 SOUTH END AVENUE, 8B",8B,10280.0,1.0,0.0,1.0,0,939.0,1990.0,2,R4,"$ 960,000",04/30/2019,2019-04-30


In [43]:
# Keep this subset
df = df[mask]

# Down from > 20k rows to < 150
df.shape

(146, 22)

In [46]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE,SALE_DATE
220,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,222,1013,NaN,R4,"181 HUDSON STREET, 6D",6D,10013.0,1.0,0.0,1.0,"7,878",1840.0,1909.0,2,R4,"$ 2,800,000",01/03/2019,2019-01-03
763,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,9022,NaN,R4,"10 LITTLE WEST STREET, 19A",19A,10004.0,1.0,0.0,1.0,0,1759.0,0.0,2,R4,"$ 2,650,000",01/07/2019,2019-01-07
996,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,212,1404,NaN,R4,"11 BEACH STREET, 1D",1D,10013.0,1.0,0.0,1.0,"11,213",2651.0,1900.0,2,R4,$ - 0,01/08/2019,2019-01-08
1276,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,16,1441,NaN,R4,"30 LITTLE WEST STREET, 7G",7G,10004.0,1.0,0.0,1.0,0,1190.0,2005.0,2,R4,"$ 1,005,000",01/09/2019,2019-01-09
1542,1,TRIBECA,02 TWO FAMILY DWELLINGS,1,132,26,NaN,S2,"75 WARREN STREET, XX",NaN,10007.0,2.0,1.0,3.0,"1,819",10117.0,1905.0,1,S2,"$ 12,950,000",01/10/2019,2019-01-10


In [51]:
# Why so few property sales in Tribeca?
# Check the date range
# It's just the first 4 months of 2019
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
df['SALE_DATE'].describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """


count                   23040
unique                    120
top       2019-01-24 00:00:00
freq                      480
first     2019-01-01 00:00:00
last      2019-04-30 00:00:00
Name: SALE_DATE, dtype: object

In [52]:
# See the Pandas Profiling Report:
# SALE_PRICE was read as strings
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [53]:
# SALE_PRICE is now a number, but hard to read in scientific notation
# Format numbers: Comma separator for thousands. Zero decimals
pd.options.display.float_format = '{:,.0f}'.format
df.describe()

,BOROUGH,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
count,"23,040","23,040","23,040",0,"23,039","23,039","23,039","23,039","23,039","23,005","23,040","23,040"
mean,3,"4,459",354,nan,"10,791",2,0,2,"3,509","1,820",2,"1,328,133"
std,1,"3,713",628,nan,"1,103",10,6,12,"22,186",488,1,"10,253,944"
min,1,1,1,nan,0,0,-148,0,0,0,1,0
25%,2,"1,341",22,nan,"10,306",0,0,1,494,"1,920",1,0
50%,3,"3,546",49,nan,"11,211",1,0,1,"1,356","1,940",1,"455,000"
75%,4,"6,674",375,nan,"11,360",2,0,2,"2,280","1,965",2,"875,000"
max,5,"16,350","9,057",nan,"11,697",750,570,755,"1,303,935","2,019",4,"850,000,000"


In [54]:
# Max SCALE_PRICE is $260 million!
# Look at this observation.
# Equivalent to: df[df['SALE_PRICE']==260000000]
df.query('SALE_PRICE == 260000000')

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
6499,1,TRIBECA,08 RENTALS - ELEVATOR APARTMENTS,2,224,1,nan,D8,34 DESBROSSES STREET,NaN,"10,013",283,3,286,"36,858","305,542","2,007",2,D8,260000000,2019-02-01


In [55]:
# How many property sales were for multiple units?
df['TOTAL_UNITS'].value_counts()

1      10204
0       5010
2       4588
3       1753
4        473
       ...  
44         1
62         1
188        1
54         1
90         1
Name: TOTAL_UNITS, Length: 102, dtype: int64

In [56]:
# Keep only the single units

# Would this be better though?
# df = df[(df.RESIDENTIAL_UNITS==1) & (df.COMMERICAL_UNITS==0)]

df = df.query('TOTAL_UNITS==1')
df.describe()

,BOROUGH,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
count,"10,204","10,204","10,204",0,"10,204","10,204","10,204","10,204","10,204","10,195","10,204","10,204"
mean,3,"4,594",667,nan,"10,825",1,0,1,"2,625","1,764",2,"1,261,156"
std,1,"3,913",776,nan,669,0,0,0,"15,073",591,1,"5,455,084"
min,1,1,1,nan,0,0,0,1,0,0,1,0
25%,2,"1,200",38,nan,"10,304",1,0,1,876,"1,924",1,0
50%,3,"3,692",244,nan,"11,209",1,0,1,"1,234","1,950",1,"500,000"
75%,4,"7,008","1,115",nan,"11,362",1,0,1,"1,666","1,991",2,"915,000"
max,5,"16,319","9,057",nan,"11,694",1,1,1,"322,160","2,018",4,"239,958,219"


In [57]:
# Now max sales price is $39 million
# Look at this observation
# It's huge, over 8,000 square feet
# Maybe it's legit, just a huge, expensive condo
df.query('SALE_PRICE == 39285000')

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
9236,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1144,nan,R4,"70 VESTRY STREET, 12S",12S,"10,013",1,0,1,0,"8,346","2,016",2,R4,39285000,2019-02-15


In [58]:
# The min sales price is $0

# There's a glossary here: 
# https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page

# It says:

# A $0 sale indicates that there was a transfer of ownership without a 
# cash consideration. There can be a number of reasons for a $0 sale including 
# transfers of ownership from parents to children. 

# How often did $0 sales occur in this subset of the data?
len(df.query('SALE_PRICE == 0'))

2962

In [59]:
# 75% percentile for gross square feet is 2,500
# But the max is over 39,000 square feet.

# Look at property sales for > 5,000 square feet

# We see the same condo we looked at earlier,
# Plus two property sales at 39,567 square feet.
# TOTAL_UNITS = 1, but BUILDING_CLASS_CATEGORY = 2-10 UNIT RESIDENTIAL
# Is this dirty data?

df.query('GROSS_SQUARE_FEET > 5000')

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,nan,R1,"20 EAST 65TH STREET, B",B,"10,065",1,0,1,0,"6,406",0,2,R1,0,2019-01-01
18,2,MORRIS PARK/VAN NEST,33 EDUCATIONAL FACILITIES,4,4090,19,nan,W6,1196 PIERCE AVENUE,NaN,"10,461",0,1,1,"23,500","12,170","1,954",4,W6,0,2019-01-01
23,2,PELHAM PARKWAY SOUTH,32 HOSPITAL AND HEALTH FACILITIES,4,4205,3,nan,I1,1301 MORRIS PARK AVENUE,NaN,"10,461",0,1,1,"215,194","217,000","2,008",4,I1,0,2019-01-01
191,5,TODT HILL,01 ONE FAMILY DWELLINGS,1,894,155,nan,A3,5 CROMWELL CIRCLE,NaN,"10,304",1,0,1,"9,927","5,048","2,004",1,A3,2425000,2019-01-02
218,1,MIDTOWN WEST,45 CONDO HOTELS,4,1006,1302,nan,RH,"1335 AVENUE OF THE AMERICAS, TIMES",TIMES,"10,019",0,1,1,"91,625","44,812","1,963",4,RH,0,2019-01-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22788,3,CROWN HEIGHTS,34 THEATRES,4,1178,24,nan,J1,558 ST JOHNS PLACE,NaN,"11,238",0,1,1,"6,454","6,301","1,910",4,J1,0,2019-04-29
22830,4,DOUGLASTON,29 COMMERCIAL GARAGES,4,8165,23,nan,GW,232-10 NORTHERN BOULEVARD,NaN,"11,363",0,1,1,"14,695","6,220","1,995",4,GW,2700000,2019-04-29
22878,1,CIVIC CENTER,12 CONDOS - WALKUP APARTMENTS,2,153,1169,nan,R4,"49-51 CHAMBERS STREET, 11F",11F,"10,007",1,0,1,"18,919","231,379","1,910",2,R2,1750000,2019-04-30
22896,1,SOHO,13 CONDOS - ELEVATOR APARTMENTS,2,477,1187,nan,R4,"565 BROOME STREET, S12A",S12A,"10,013",1,0,1,"21,487","322,160","2,016",2,R4,3600000,2019-04-30


In [60]:
# What are the building class categories?
# How frequently does each occur?

df['BUILDING_CLASS_CATEGORY'].value_counts()

01 ONE FAMILY DWELLINGS                       4901
13 CONDOS - ELEVATOR APARTMENTS               3236
04 TAX CLASS 1 CONDOS                          403
15 CONDOS - 2-10 UNIT RESIDENTIAL              378
44 CONDO PARKING                               329
12 CONDOS - WALKUP APARTMENTS                  240
22 STORE BUILDINGS                             115
43 CONDO OFFICE BUILDINGS                       76
30 WAREHOUSES                                   76
47 CONDO NON-BUSINESS STORAGE                   75
46 CONDO STORE BUILDINGS                        59
29 COMMERCIAL GARAGES                           53
27 FACTORIES                                    42
21 OFFICE BUILDINGS                             35
45 CONDO HOTELS                                 31
37 RELIGIOUS FACILITIES                         23
16 CONDOS - 2-10 UNIT WITH COMMERCIAL UNIT      21
28 COMMERCIAL CONDOS                            18
11A CONDO-RENTALS                               18
33 EDUCATIONAL FACILITIES      

In [61]:
# Keep subset of rows:
# Sale price more than $0, 
# Building class category = Condos - Elevator Apartments

mask = (df['SALE_PRICE'] > 0) & (df['BUILDING_CLASS_CATEGORY'] == '13 CONDOS - ELEVATOR APARTMENTS')
df = df[mask]

# Down to 106 rows
df.shape

(2412, 21)

In [62]:
# Scatter plot
import plotly.express as px
px.scatter(df, x='GROSS_SQUARE_FEET', y='SALE_PRICE')

In [63]:
# With OLS (Ordinary Least Squares) trendline,
# The outliers influence the "line of best fit"
px.scatter(df, x='GROSS_SQUARE_FEET', y='SALE_PRICE', trendline='ols')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [64]:
# Look at sales for more than $35 million

# All are at 70 Vestry Street
# All but one have the same SALE_PRICE & SALE_DATE
# Was the SALE_PRICE for each? Or in total?
# Is this dirty data?

df.query('SALE_PRICE > 35000000')

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
3995,1,MIDTOWN WEST,13 CONDOS - ELEVATOR APARTMENTS,2,1030,1082,nan,R4,"220 CENTRAL PARK SOUTH, 50",50,"10,019",1,0,1,0,"23,029","2,015",2,R4,239958219,2019-01-23
4779,1,UPPER EAST SIDE (59-79),13 CONDOS - ELEVATOR APARTMENTS,2,1375,1437,nan,R4,"520 PARK AVENUE, DPH54",DPH54,"10,022",1,0,1,0,"9,138","2,015",2,R4,64250000,2019-01-25
8370,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1105,nan,R4,"70 VESTRY STREET, 3C",3C,"10,013",1,0,1,0,"1,670","2,016",2,R4,36681561,2019-02-12
8371,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1123,nan,R4,"70 VESTRY STREET, 6C",6C,"10,013",1,0,1,0,"1,906","2,016",2,R4,36681561,2019-02-12
8372,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1124,nan,R4,"70 VESTRY STREET, 6D",6D,"10,013",1,0,1,0,"2,536","2,016",2,R4,36681561,2019-02-12
8373,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1125,nan,R4,"70 VESTRY STREET, 6E",6E,"10,013",1,0,1,0,"2,965","2,016",2,R4,36681561,2019-02-12
8374,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1126,nan,R4,"70 VESTRY STREET, 6F",6F,"10,013",1,0,1,0,"2,445","2,016",2,R4,36681561,2019-02-12
8375,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1127,nan,R4,"70 VESTRY STREET, 7A",7A,"10,013",1,0,1,0,"2,844","2,016",2,R4,36681561,2019-02-12
8376,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1128,nan,R4,"70 VESTRY STREET, 7B",7B,"10,013",1,0,1,0,"3,242","2,016",2,R4,36681561,2019-02-12
8377,1,TRIBECA,13 CONDOS - ELEVATOR APARTMENTS,2,223,1129,nan,R4,"70 VESTRY STREET, 7C",7C,"10,013",1,0,1,0,"1,906","2,016",2,R4,36681561,2019-02-12


In [65]:
# Make a judgment call:
# Keep rows where sale price was < $35 million
df = df.query('SALE_PRICE < 35000000')

# Down to 90 rows
df.shape

(2394, 21)

In [68]:
columns = ['NEIGHBORHOOD', 
           'BUILDING_CLASS_CATEGORY', 
           'ADDRESS', 
           'APARTMENT_NUMBER', 
           'ZIP_CODE', 
           'GROSS_SQUARE_FEET',  
           'YEAR_BUILT', 
           'SALE_PRICE', 
           'SALE_DATE']

df[columns].to_csv('C:\\Users\\Nkiru\\Documents\\ARM\\tribeca.csv', index=False)